## Generate data 

In [1]:
import sys
sys.path.append('/'.join(sys.path[0].split('/')[:-1]))

import os
import xarray as xr
import numpy as np
import time
import matplotlib.pyplot as plt
import healpy as hp

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


In [2]:
datadir = "../data/healpix/5.625deg_nearest/"

lr=1e-4
dr=0
batch_size=128
patience=3
train_years=('1979', '2015')
valid_years=('2016', '2016')
test_years=('2017', '2018')
gpu=1
iterative=False

vars = ['z', 't']
kernel_size = 5

In [3]:
z = xr.open_mfdataset(f'{datadir}geopotential_500/*.nc', combine='by_coords')
t = xr.open_mfdataset(f'{datadir}temperature_850/*.nc', combine='by_coords')
ds = xr.merge([z, t], compat='override')

In [4]:
def create_iterative_observations_healpix(ds, lead_time, max_lead_time, nb_timesteps, test_years, nodes):
    
    lead_times = np.arange(lead_time, max_lead_time + lead_time, lead_time)

    data = ds.to_array(dim='level', name='Dataset').transpose('time', 'node', 'level')
    n_samples = data.isel(time=slice(0, -nb_timesteps*lead_time)).shape[0] - max_lead_time

    obs_list = []
    
    print('Generating observations list...')
    for lead in lead_times:
        obs_list.append(data.isel(time=slice(lead, lead + n_samples)).isel(level=slice(0, 2)))

    #observations_numpy = np.array(obs_list)
    observations_joint = xr.concat(obs_list, dim='lead_time')
    #return observations_numpy
    print('Obtaining coordinates...')
    # Lat lon coordinates
    nside = int(np.sqrt(nodes/12))
    out_lon, out_lat = hp.pix2ang(nside, np.arange(nodes), lonlat=True)
    
    print('Generate set of times to study', end='\n')
    # Actual times
    start = np.datetime64(test_years[0], 'h') + np.timedelta64(lead_time, 'h')
    stop = start + np.timedelta64(n_samples, 'h')
    times = np.arange(start, stop)

    # Variables
    var_dict_out = {var: None for var in ['z', 't']}

    das = [];
    lev_idx = 0
   
    for var, levels in var_dict_out.items():
        das.append(observations_joint.isel(level=lev_idx).rename(var))
        lev_idx +=1
        
        """
        
        
        if levels is None:            
            das.append(xr.DataArray(
                observations_numpy[:, :, :, lev_idx],
                dims=['lead_time', 'time', 'node'],
                coords={'lead_time': lead_times, 'time': times, 'node': np.arange(nodes)},
                name=var
            ))
            lev_idx += 1
        # never this case...
        else:
            nlevs = len(levels)
            das.append(xr.DataArray(
                observations_numpy[:, :, :, lev_idx:lev_idx+nlevs],
                dims=['lead_time', 'time', 'node', 'level'],
                coords={'lead_time': lead_times, 'time': valid_time, 'node': nodes, 'level': nlevs},
                name=var
            ))
            lev_idx += nlevs
       """
    
    print('\nGenerate observation...')
    observation_ds = xr.merge(das, compat='override').reset_coords(names='level', drop=True)
    observation_ds = observation_ds.assign_coords({'lat': out_lat, 'lon': out_lon})
    return observation_ds


In [5]:
nodes = 12*16*16
max_lead_time = 5*24
lead_time = 6
out_features = 2
nb_timesteps = 2
nside = int(np.sqrt(nodes/12))

In [6]:
#obs = create_iterative_observations_hp("../data/equiangular/5.625deg/", test_years, lead_time, max_lead_time, nside, nb_timesteps=2)

In [7]:
obs = create_iterative_observations_healpix(ds, lead_time, max_lead_time, nb_timesteps, test_years, nodes)

Generating observations list...
Obtaining coordinates...
Generate set of times to study

Generate observation...


In [8]:
obs

<xarray.Dataset>
Dimensions:  (lat: 3072, lead_time: 20, lon: 3072, node: 3072, time: 350622)
Coordinates:
  * lat      (lat) float64 87.08 87.08 87.08 87.08 ... -87.08 -87.08 -87.08
  * lon      (lon) float64 45.0 135.0 225.0 315.0 ... 45.0 135.0 225.0 315.0
  * time     (time) datetime64[ns] 1979-01-01T06:00:00 ... 2018-12-31T11:00:00
  * node     (node) int64 0 1 2 3 4 5 6 7 ... 3065 3066 3067 3068 3069 3070 3071
Dimensions without coordinates: lead_time
Data variables:
    z        (lead_time, time, node) float32 dask.array<chunksize=(1, 350622, 3072), meta=np.ndarray>
    t        (lead_time, time, node) float32 dask.array<chunksize=(1, 350622, 3072), meta=np.ndarray>

In [9]:
obs.to_netcdf(datadir + 'observations.nc')

### Use new functions from Iciar

In [4]:
from modules.test import create_iterative_observations_hp

In [5]:
nodes = 12*16*16
max_lead_time = 5*24
lead_time = 6
out_features = 2
nb_timesteps = 2
nside = int(np.sqrt(nodes/12))

In [10]:
def create_iterative_observations_hp(input_dir, test_years, lead_time, max_lead_time, nside, nb_timesteps=2):
    z500 = xr.open_mfdataset(f'{input_dir}geopotential_500/*.nc', combine='by_coords').sel(time=slice(*test_years))
    t850 = xr.open_mfdataset(f'{input_dir}temperature_850/*.nc', combine='by_coords').sel(time=slice(*test_years))

    test_data = xr.merge([z500, t850], compat='override')


    n_samples = test_data.isel(time=slice(0, -nb_timesteps*lead_time)).dims['time'] - max_lead_time
    nb_iter = max_lead_time // lead_time
    n_pixels = 12*(nside**2)
    print(n_samples)

    # Lead times
    lead_times = np.arange(lead_time, max_lead_time + lead_time, lead_time)

    # Lat lon coordinates
    out_lon, out_lat = hp.pix2ang(nside, np.arange(n_pixels), lonlat=True)

    # Actual times
    start = np.datetime64(test_years[0], 'h') + np.timedelta64(lead_time, 'h')
    stop = start + np.timedelta64(n_samples, 'h')
    times = np.arange(start, stop, 1)

    # Variables
    data_vars = ['z', 't']
    var_dict_out = {var: None for var in data_vars}
    
    data = np.zeros((2, nb_iter, n_samples, 3072))
    
    #data = np.zeros((2, nb_iter, n_samples, 32, 64))
    for i in range(nb_iter):
        #data[0, i, :, :, :] = test_data.z.isel(time=slice(lead_time*(i+1), lead_time*(i+1) + n_samples)).values
        #data[1, i, :, :, :] = test_data.t.isel(time=slice(lead_time*(i+1), lead_time*(i+1) + n_samples)).values
        
        data[0, i, :, :] = test_data.z.isel(time=slice(lead_time*(i+1), lead_time*(i+1) + n_samples)).values
        data[1, i, :, :] = test_data.t.isel(time=slice(lead_time*(i+1), lead_time*(i+1) + n_samples)).values


    das = [];
    lev_idx = 0
    for var in data_vars:       
        das.append(xr.DataArray(
         data[lev_idx, :, :, :],
         dims=['lead_time', 'time', 'node'],
         coords={'lead_time': lead_times, 'time': times, 'node': np.arange(n_pixels)},
         name=var
     ))
        lev_idx += 1
    observations = xr.merge(das)
    #observations = observations.assign_coords({'lat': out_lat, 'lon': out_lon})

    return observations

In [11]:
observations = create_iterative_observations_hp('../data/healpix/5.625deg_nearest/', test_years, lead_time, max_lead_time, nside, nb_timesteps=2)

17388


In [13]:
observations

<xarray.Dataset>
Dimensions:    (lead_time: 20, node: 3072, time: 17388)
Coordinates:
  * lead_time  (lead_time) int64 6 12 18 24 30 36 42 ... 90 96 102 108 114 120
  * time       (time) datetime64[ns] 2017-01-01T06:00:00 ... 2018-12-26T17:00:00
  * node       (node) int64 0 1 2 3 4 5 6 ... 3065 3066 3067 3068 3069 3070 3071
Data variables:
    z          (lead_time, time, node) float64 5.761e+04 5.764e+04 ... 5.751e+04
    t          (lead_time, time, node) float64 291.4 290.7 291.1 ... 290.5 290.3

In [15]:
n_pixels = 12*(nside**2)
out_lon, out_lat = hp.pix2ang(nside, np.arange(n_pixels), lonlat=True)

In [18]:
observations = observations.assign_coords({'lat': ('node', out_lat), 'lon': ('node', out_lon)})

In [19]:
observations.to_netcdf('../data/healpix/predictions/' + 'observations_nearest.nc')